In [1]:
import pandas as pd
df = pd.read_csv("data.csv")
df.head()

,Category,Message
0,1,Last summer I had an appointment to get new ti...
1,2,"Friendly staff, same starbucks fair you get an..."
2,1,The food is good. Unfortunately the service is...
3,2,Even when we didn't have a car Filene's Baseme...
4,2,"Picture Billy Joel's \""Piano Man\"" DOUBLED mix..."


# Import Dependencies

In [2]:
import re
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Embedding, Dropout
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Tokenize the words and pad the sequences

In [13]:
tokenizer = Tokenizer(num_words=1000, split=" ")
tokenizer.fit_on_texts(df['Message'].values)

X = tokenizer.texts_to_sequences(df['Message'].values)
X = pad_sequences(X) # padding our text vector so they all have the same length
X[:5]

array([[  0,   0,   0, ..., 554, 685,  86],
       [  0,   0,   0, ...,  72,  43, 188],
       [  0,   0,   0, ...,  98,   4, 879],
       [  0,   0,   0, ...,   3, 214, 370],
       [  0,   0,   0, ...,  46,   4,  49]])

In [16]:
y = pd.get_dummies(df['Category']).values
[print(df['Category'][i],y[i]) for i in range(0,5)]

1 [1 0]
2 [0 1]
1 [1 0]
2 [0 1]
2 [0 1]


[None, None, None, None, None]

# Split the data

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [22]:
X_train.shape

(30399, 844)

In [23]:
y_train.shape

(30399, 2)

# Create a Bidirectional RNN  

In [30]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(
        input_dim=1000,
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2)
])

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, None, 64)          64000     
_________________________________________________________________
bidirectional_5 (Bidirection (None, 128)               66048     
_________________________________________________________________
dense_10 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_11 (Dense)             (None, 2)                 130       
Total params: 138,434
Trainable params: 138,434
Non-trainable params: 0
_________________________________________________________________


In [31]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [32]:
history = model.fit(X_train, y_train, epochs=3, batch_size=32, verbose=1)

Epoch 1/3
290/950 [========>.....................] - ETA: 38:46 - loss: nan - accuracy: 0.5024

KeyboardInterrupt: 